In [4]:
# When working in AI LRZ
%cd ~/cma/CMA_Fairness_v2

/dss/dsshome1/0C/ra93lal2/cma/CMA_Fairness_v2


/dss/dsshome1/0C/ra93lal2/.local/share/virtualenvs/CMA_Fairness_v2-3j10GkSs/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/dss/dsshome1/0C/ra93lal2/.local/share/virtualenvs/CMA_Fairness_v2-3j10GkSs/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
import pandas as pd

# Configuration

In [6]:
# Auto-reload the custom package
%load_ext autoreload
%autoreload 1
%aimport fairness_multiverse


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
RUN_TO_ANALYSE = "11"

In [8]:
PREFIX_SETTINGS = "sett_"
PREFIX_EVAL = "sett_eval_"
PREFIX_PERFORMANCE = "perf_"
PREFIX_FAIRNESS = "fair_"

In [9]:
from pathlib import Path

OUTPUT_DIR = Path(".") / "output"

# Directory that will contain outputs from analysis
ANALYSIS_OUTPUT_DIR = OUTPUT_DIR / "analyses" / (str(RUN_TO_ANALYSE))
ANALYSIS_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

DATA_DIR = OUTPUT_DIR / "runs" / str(RUN_TO_ANALYSE) / "data"

In [10]:
main_fairness_metric = "fair_main_equalized_odds_difference"

## Data Loading

In [11]:
df_agg_raw = pd.read_csv(DATA_DIR / f"agg_{RUN_TO_ANALYSE}_run_outputs.csv.gz")

In [12]:
import json

df_settings = pd.json_normalize(
    df_agg_raw["universe_settings"].apply(json.loads)
).add_prefix(
    PREFIX_SETTINGS
)

df_agg_full = df_settings.join(df_agg_raw)
df_agg_full.head()

,sett_cutoff,sett_eval_fairness_grouping,sett_exclude_features,sett_exclude_subgroups,sett_model,Unnamed: 0,run_no,universe_id,universe_settings,execution_time,...,perf_grp_precision_0,perf_grp_precision_1,perf_grp_false positive rate_0,perf_grp_false positive rate_1,perf_grp_false negative rate_0,perf_grp_false negative rate_1,perf_grp_selection rate_0,perf_grp_selection rate_1,perf_grp_count_0,perf_grp_count_1
0,quantile_0.1,majority-minority,nationality,drop-non-german,rf,0,11,338397ae0ea0b5f086116cbd3197dacc,"{""cutoff"": ""quantile_0.1"", ""eval_fairness_grou...",39.464597,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,quantile_0.1,nationality-all,nationality,drop-non-german,rf,1,11,338397ae0ea0b5f086116cbd3197dacc,"{""cutoff"": ""quantile_0.1"", ""eval_fairness_grou...",39.464597,...,0.120519,0.139268,0.909971,0.930918,0.043074,0.010722,0.915385,0.938629,20540.0,69170.0
2,quantile_0.25,majority-minority,nationality,drop-non-german,rf,2,11,338397ae0ea0b5f086116cbd3197dacc,"{""cutoff"": ""quantile_0.25"", ""eval_fairness_gro...",39.464597,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,quantile_0.25,nationality-all,nationality,drop-non-german,rf,3,11,338397ae0ea0b5f086116cbd3197dacc,"{""cutoff"": ""quantile_0.25"", ""eval_fairness_gro...",39.464597,...,0.138577,0.165347,0.700913,0.730318,0.134713,0.049781,0.719864,0.759375,20540.0,69170.0
4,quantile_0.1,majority-minority,sex,keep-all,elasticnet,4,11,418d84cd0641196cf291bbbd3833bfcd,"{""cutoff"": ""quantile_0.1"", ""eval_fairness_grou...",234.207067,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
rows, columns = df_agg_full.shape
print(f"The data has N = {rows} rows and N = {columns} columns.")

In [ ]:
cols_settings = list(df_agg_full.columns[df_agg_full.columns.str.startswith(PREFIX_SETTINGS)])
cols_eval = list(df_agg_full.columns[df_agg_full.columns.str.startswith(PREFIX_EVAL)])
cols_non_eval = list(set(cols_settings) - set(cols_eval))
cols_performance = list(df_agg_full.columns[df_agg_full.columns.str.startswith(PREFIX_PERFORMANCE)])
cols_fairness = list(df_agg_full.columns[df_agg_full.columns.str.startswith(PREFIX_FAIRNESS)])


In [ ]:
df_agg_full["universe_id"]

In [ ]:
drop_mask = df_agg_full["sett_exclude_subgroups"].isin(['drop-non-german'])
print(f"Dropping N = {drop_mask.sum()} rows, keeping N = {(~drop_mask).sum()}")

df_agg_full = df_agg_full[~drop_mask]

In [ ]:
df_agg_full["sett_exclude_subgroups"].unique()

In [ ]:
df_agg_full["sett_eval_fairness_grouping"].unique()

In [ ]:
#df_agg_full["sett_eval_exclude_subgroups"].unique()

In [ ]:
#df_agg_full["sett_eval_on_subset"].unique()